In [1]:
import pandas as pd
import sqlite3

In [2]:
connect = sqlite3.connect('../../datasets/checking-logs.sqlite')

In [3]:
query = """
WITH users_with_both_periods AS (
    SELECT uid, first_view_ts
    FROM test
    WHERE labname != 'project1'
        AND first_view_ts IS NOT NULL
        AND EXISTS (SELECT 1 FROM test t_before WHERE t_before.uid = test.uid 
            AND t_before.first_commit_ts < test.first_view_ts) 
        AND EXISTS (SELECT 1 FROM test t_after WHERE t_after.uid = test.uid 
            AND t_after.first_commit_ts >= test.first_view_ts))
SELECT 
    CASE 
        WHEN t.first_commit_ts < u.first_view_ts THEN 'before'
        ELSE 'after' 
    END AS time,AVG((strftime('%s', t.first_commit_ts) - d.deadlines) / 3600) AS avg_diff
FROM test t
JOIN deadlines d ON t.labname = d.labs
JOIN users_with_both_periods u ON t.uid = u.uid
WHERE t.labname != 'project1'
GROUP BY time;
"""
test_results = pd.io.sql.read_sql(query, connect)
test_results

,time,avg_diff
0,after,-96.354839
1,before,-61.068493


In [4]:
query = '''
WITH users_with_both_periods AS (
    SELECT uid
    FROM control
    WHERE labname != 'project1'
    GROUP BY uid
    HAVING SUM(CASE WHEN first_commit_ts < first_view_ts THEN 1 ELSE 0 END) > 0
       AND SUM(CASE WHEN first_commit_ts >= first_view_ts THEN 1 ELSE 0 END) > 0),
time_periods AS (
    SELECT 
        CASE 
            WHEN c.first_commit_ts < c.first_view_ts THEN 'before'
            ELSE 'after'
        END AS time,
        (strftime('%s', c.first_commit_ts) - d.deadlines) / 3600 AS diff_hours
    FROM control c
    JOIN deadlines d ON c.labname = d.labs
    JOIN users_with_both_periods u ON c.uid = u.uid
    WHERE c.labname != 'project1')
SELECT 
    time,
    AVG(diff_hours) AS avg_diff
FROM time_periods
GROUP BY time
UNION ALL
SELECT 'after', NULL WHERE NOT EXISTS (SELECT 1 FROM time_periods WHERE time = 'after')
UNION ALL
SELECT 'before', NULL WHERE NOT EXISTS (SELECT 1 FROM time_periods WHERE time = 'before')
ORDER BY time DESC;
'''
control_results = pd.io.sql.read_sql(query, connect)
control_results

,time,avg_diff
0,before,-99.464286
1,after,-117.636364


In [5]:
connect.close()